In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten , Conv2D , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
train_data_dir = 'data/'

In [3]:
img_width, img_height = 224, 224
batch_size = 16
epochs = 100
num_classes = 5

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 2931 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
x = base_model.output
x = Conv2D(2048 , activation='relu' , kernel_size=(3 , 3))(x)
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(512 , activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128 , activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(32 , activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/100
183/183 [==============================] - 34s 151ms/step - loss: 1.0772 - accuracy: 0.6669 - val_loss: 0.8362 - val_accuracy: 0.7056
Epoch 2/100
183/183 [==============================] - 26s 142ms/step - loss: 0.8251 - accuracy: 0.7081 - val_loss: 0.7803 - val_accuracy: 0.6972
Epoch 3/100
183/183 [==============================] - 26s 142ms/step - loss: 0.7919 - accuracy: 0.7149 - val_loss: 0.7257 - val_accuracy: 0.7319
Epoch 4/100
183/183 [==============================] - 26s 142ms/step - loss: 0.7623 - accuracy: 0.7249 - val_loss: 0.7762 - val_accuracy: 0.7319
Epoch 5/100
183/183 [==============================] - 26s 142ms/step - loss: 0.7635 - accuracy: 0.7389 - val_loss: 0.6953 - val_accuracy: 0.7486
Epoch 6/100
183/183 [==============================] - 26s 142ms/step - loss: 0.7012 - accuracy: 0.7516 - val_loss: 0.7462 - val_accuracy: 0.7347
Epoch 7/100
183/183 [==============================] - 26s 142ms/step - loss: 0.6885 - accuracy: 0.7516 - val_loss: 0.7173 -

In [12]:
model.save('model.h5')

In [13]:
from pickle import dump
with open('classes.h5' , 'wb') as f:
    dump(train_generator.class_indices , f)